In [ ]:
!nvidia-smi

Mon May 15 21:48:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    51W / 400W |   7991MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Requirements
!pip install nltk
!pip install transformers
!pip install shutup
!pip install pytorch-lightning
!pip install torchMetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import torch
import torch.nn as nn
import pytorch_lightning as pl
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader, random_split
from pylab import rcParams
from tqdm.auto import tqdm
from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup
#from torchmetrics.functional import f1, accuracy
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from matplotlib import rc

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
pl.seed_everything(RANDOM_SEED)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#df = pd.read_csv('drive/MyDrive/training_images/training.csv', sep='\t')
df = pd.read_csv('drive/MyDrive/test.tsv', sep='\t')
#df = pd.read_csv('sample_data/training.csv', sep='\t')

In [ ]:
df.rename(columns={'Text Transcription':'text'}, inplace=True)

In [ ]:
df

,file_name,misogynous,shaming,stereotype,objectification,violence,text
0,15236.jpg,0,0,0,0,0,FACEBOOK SINGLES GROUPS BELIKE WHEN A NEW WOMA...
1,15805.jpg,1,0,1,1,0,"SO, IF YOU'RE A FEMINIST HOW CAN YOU EAT DAIRY?"
2,16254.jpg,0,0,0,0,0,WHEN A CUTE GIRL LEFT YOUR MESSAGE ON SEEN
3,16191.jpg,1,0,1,1,0,Photographing something you want to show every...
4,15952.jpg,0,0,0,0,0,HEY BABE CAN YOU MAKE ME A SANDWICH? Hey babe ...
...,...,...,...,...,...,...,...
995,15591.jpg,1,0,1,1,0,IT'S NOT YOUR FAULT You didn't design the dres...
996,15049.jpg,0,0,0,0,0,THINK ABOUT HOW MUCH BETTER HER SKIN IS BREATH...
997,15363.jpg,1,0,1,1,0,THE STEREOTYPES ARE TRUE F SHE DOES HAVE A TIG...
998,15199.jpg,0,0,0,0,0,DRAWS NAKED PICTURES OF BLACK WOMEN 00 0000 GE...


In [ ]:
from nltk import word_tokenize

def preprocess_text(text):

  # Remove extra whitespaces
  text = text.strip()

  # Remove twitter usernames, web addresses
  text = text = re.sub(r"#[\w\d]*|@[.]?[\w\d]*[\'\w*]*|https?:\/\/\S+\b|"r"www\.(\w+\.)+\S*|", '', text)

  # Remove html tags
  text = re.sub(re.compile('<.*?>'), ' ', text)

  # Remove unwanted characters
  text = word_tokenize(text)
  text = ' '.join(word for word in text if word.isalpha() or word.isnumeric() or word.isalnum())
  
  return text

In [ ]:
df.text = np.array([preprocess_text(text) for text in df.text])

In [ ]:
MAX_LEN = 64

'YOU RAPE HER SHE KNOWS SHE ASKED FOR IT'

In [ ]:
# import pickle 

# with open('/content/drive/MyDrive/visual_embeds_train.pkl', 'rb') as f:
#   visual_embeds_mami = pickle.load(f)


import pickle 

with open('/content/drive/MyDrive/visual_embeds_test.pkl', 'rb') as f:
  visual_embeds_mami = pickle.load(f)

In [ ]:
import torch
torch.__version__

torch.cuda.is_available()

(1000, 4)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(len(visual_embeds_mami))
visual_embeds_mami[0].shape

In [ ]:
from transformers import VisualBertModel, VisualBertConfig

configuration = VisualBertConfig.from_pretrained('uclanlp/visualbert-vqa', visual_embedding_dim=1024)
model = VisualBertModel(configuration)
model = model.to(device)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

True

In [ ]:
visual_embeds_mami[:1]
index = 2

In [ ]:
# tokens = tokenizer(df_train.text[index], padding='max_length', max_length=64)

tokens = tokenizer.encode_plus(
      df.text[index],
      add_special_tokens=True,
      max_length=64,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
)

input_ids = torch.tensor(tokens["input_ids"]).to(device).flatten()
attention_mask = torch.tensor(tokens["attention_mask"]).to(device).flatten()

visual_embeds = visual_embeds_mami[index].to(device)
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.float).to(device)

<ipython-input-385-0365fb3b5d09>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(tokens["input_ids"]).to(device).flatten()
<ipython-input-385-0365fb3b5d09>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(tokens["attention_mask"]).to(device).flatten()


In [ ]:
print(input_ids.shape)
print(attention_mask.shape)
print(visual_embeds.shape)
print(visual_attention_mask.shape)

torch.Size([64])
torch.Size([64])
torch.Size([100, 1024])
torch.Size([100])


In [ ]:
outputs = model(input_ids=input_ids.unsqueeze(0),
                attention_mask=attention_mask.unsqueeze(0),
                visual_embeds=visual_embeds.unsqueeze(0),
                visual_attention_mask=visual_attention_mask.unsqueeze(0),
                )

# last_hidden_state = outputs.last_hidden_state
print(outputs.last_hidden_state.shape)
print(outputs.pooler_output.shape)

torch.Size([1, 164, 768])
torch.Size([1, 768])


In [ ]:
for i in range(len(visual_embeds_mami)):
  visual_embeds_mami[i] = visual_embeds_mami[i].to('cpu')

In [ ]:
class MAMIDataset(Dataset):

  def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizer, max_len: int, visual_embeds):
    self.tokenizer = tokenizer
    self.data = data
    self.max_len = max_len
    self.visual_embeds = visual_embeds
    
  
  def __len__(self):
    print(len(self.data))
    return len(self.data)

  def __getitem__(self, index):

    data_row = self.data.iloc[index]
    text = data_row.text
    labels = data_row.misogynous

    tokens = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    input_ids = torch.tensor(tokens["input_ids"]).flatten()
    attention_mask = torch.tensor(tokens["attention_mask"]).flatten()

    visual_embedding = self.visual_embeds[index].to('cpu')
    visual_attention_mask = torch.ones(visual_embedding.shape[:-1], dtype=torch.float)
    visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long)

    return dict(
      input_ids=input_ids,
      attention_mask=attention_mask,
      visual_embedding=visual_embedding,
      visual_attention_mask=visual_attention_mask,
      visual_token_type_ids=visual_token_type_ids,
      labels=torch.tensor(labels).float()
    )

In [ ]:
dataset = MAMIDataset(
  df,
  tokenizer,
  MAX_LEN,
  visual_embeds_mami
)





In [ ]:
train_dataset, val_dataset = train_test_split(dataset, test_size=0.10)

1000
1000
1000
1000


<ipython-input-389-0aba3688184a>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(tokens["input_ids"]).flatten()
<ipython-input-389-0aba3688184a>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(tokens["attention_mask"]).flatten()


In [ ]:
class MemotionDataModule(pl.LightningDataModule):

  def __init__(self, train_dataset, val_dataset, tokenizer, batch_size=32, max_len=64):
    super().__init__()
    self.batch_size = batch_size
    # self.df = df
    self.train_dataset = train_dataset
    self.val_dataset = val_dataset
    self.tokenizer = tokenizer
    self.max_len = max_len
    # self.visual_embeds_mami = visual_embeds_mami
  
  # def setup(self, stage=None):
  #   self.dataset = MAMIDataset(self.df, self.tokenizer, self.max_len, self.visual_embeds_mami)
  #   self.train_dataset, self.val_dataset = train_test_split(self.dataset, test_size=0.1)
  
  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size = self.batch_size,
        shuffle=True,
        num_workers=0
    )
  
  def val_dataloader(self):
    return DataLoader(
        self.val_dataset,
        batch_size = self.batch_size,
        num_workers=0
    )
  
  def test_dataloader(self):
    return DataLoader(
        self.val_dataset,
        batch_size = self.batch_size,
        num_workers=0
    )

In [ ]:
class MemesClassifier(pl.LightningModule):
  
  def __init__(self, n_classes, n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.configuration = VisualBertConfig.from_pretrained('uclanlp/visualbert-vqa-coco-pre', visual_embedding_dim=1024)
    self.model = VisualBertModel(configuration)
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.CrossEntropyLoss()
    self.dropout = nn.Dropout(0.2)
    self.classifier = nn.Linear(self.model.config.hidden_size, n_classes)
  
  
  def forward(self, input_ids, attention_mask, visual_embeds, visual_attention_mask, visual_token_type_ids, labels=None):
    output = self.model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        visual_embeds=visual_embeds,
                        visual_attention_mask=visual_attention_mask,
                        visual_token_type_ids=visual_token_type_ids)
    
    # output = self.dropout(output.pooler_output)
    output = self.classifier(output.pooler_output)

    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)

    return loss, output
  
  def training_step(self, batch, batch_idx):
    
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    visual_embeds = batch['visual_embedding'].to(device)
    visual_attention_mask = batch['visual_attention_mask'].to(device)
    visual_token_type_ids = batch['visual_token_type_ids'].to(device)

    labels = batch['labels'].type(torch.LongTensor).to(device)
    
    loss, outputs = self(input_ids, attention_mask, visual_embeds, visual_attention_mask, visual_token_type_ids, labels)
    self.log('train_loss', loss, prog_bar=True, logger=True)

    return {"loss":loss, 'predictions':outputs, 'labels':labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    visual_embeds = batch['visual_embedding']
    visual_attention_mask = batch['visual_attention_mask']
    visual_token_type_ids = batch['visual_token_type_ids'].to(device)
    labels = batch['labels'].type(torch.LongTensor).to(device)
    
    loss, outputs = self(input_ids, attention_mask, visual_embeds, visual_attention_mask, visual_token_type_ids, labels)
    self.log('val_loss', loss, prog_bar=True, logger=True)

    return loss
  
  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=5e-5)

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=self.n_warmup_steps,
        num_training_steps=self.n_training_steps
    )

    return dict(
        optimizer=optimizer,
        lr_scheduler=dict(
            scheduler=scheduler,
            interval='step'
        )
    )

In [ ]:
n_classes = 2
N_EPOCHS = 10
BATCH_SIZE = 32

In [ ]:
steps_per_epoch = 9000 // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

In [ ]:
warmup_steps = total_training_steps // 10
warmup_steps, total_training_steps

(281, 2810)

In [ ]:
model = MemesClassifier(
  n_classes=n_classes,
  n_training_steps=total_training_steps,
  n_warmup_steps=0,
).to(device)


In [ ]:
checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="val_loss",
  mode="min",
)

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="memes-text")
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)
len(dataset)

10000


10000

In [ ]:
from sklearn.model_selection import KFold
import sklearn



dataset = MAMIDataset(
  df,
  tokenizer,
  MAX_LEN,
  visual_embeds_mami
)

train_dataset, val_dataset = train_test_split(dataset, test_size=0.1, random_state=RANDOM_SEED, shuffle=True)

data_module = MemotionDataModule(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    tokenizer=tokenizer,
    batch_size=BATCH_SIZE,
)

trainer = pl.Trainer(
    logger=logger,
    callbacks=[early_stopping_callback, checkpoint_callback],
    max_epochs=10
    #gpus=1,
    #progress_bar_refresh_rate=10
)

trainer.fit(model, data_module)


10000
10000
10000
10000


<ipython-input-359-0aba3688184a>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(tokens["input_ids"]).flatten()
<ipython-input-359-0aba3688184a>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(tokens["attention_mask"]).flatten()
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 5, global step 1692: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 6, global step 1974: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 2256: 'val_loss' was not in top 1


In [ ]:
trained_model = MemesClassifier.load_from_checkpoint(
  '/content/checkpoints/best-checkpoint-v6.ckpt',
  n_classes=2
)
trained_model.eval()
trained_model.freeze()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = trained_model.to(device)

predictions = []
labels = []
for item in tqdm(train_dataset):
  _, prediction = trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device),
    item["attention_mask"].unsqueeze(dim=0).to(device),
    item["visual_embedding"].unsqueeze(dim=0).to(device),
    item['visual_attention_mask'].unsqueeze(dim=0).to(device),
    item['visual_token_type_ids'].unsqueeze(dim=0).to(device)
  )
  predictions.append(prediction.flatten())
  labels.append(item["labels"].int())
  
predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()

  0%|          | 0/999 [00:00<?, ?it/s]

In [ ]:
from torch.nn.functional import softmax

_, preds = torch.max(torch.tensor(predictions), dim=1)

<ipython-input-338-6317a4a4033b>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _, preds = torch.max(torch.tensor(predictions), dim=1)


In [ ]:
preds[17]

tensor(0)

In [ ]:
from sklearn.metrics import f1_score

print("F1 macro:   {}".format(round(f1_score(labels, preds , average="macro"), 3)))
print("F1 micro:   {}".format(round(f1_score(labels, preds > 0.5, average="micro"), 3)))

F1 macro:   0.626
F1 micro:   0.649
